![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [22]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
qb = QuantBook()
spy = qb.AddEquity("SPY")
history = qb.History(qb.Securities.Keys, 360, Resolution.Daily)

# Indicator Analysis
bbdf = qb.Indicator(BollingerBands(30, 2), spy.Symbol, 360, Resolution.Daily)
bbdf.drop('standarddeviation', 1).plot()

In [1]:
qb = QuantBook()

In [4]:
from MyEnv import StockTradingEnv
from stable_baselines.a2c.a2c import A2C
import pandas as pd
import numpy as np

In [8]:
from io import StringIO
df = pd.read_csv(StringIO(qb.Download('https://raw.githubusercontent.com/iamianM/model/main/temp.csv')))

In [9]:
    env_kwargs = {
        "hmax": 1.0, 
        'model_name':'TSLA',
        'mode': 'train',
        "initial_amount": 50000, 
        "buy_cost_pct": lambda x: 0.01, 
        "sell_cost_pct": lambda x: 0.01, 
        "tech_indicator_list": [], 
        "print_verbosity":1000,
        "terminal": True,
    }
    env = StockTradingEnv(df, **env_kwargs)

In [10]:
    model = A2C(
        policy="CnnPolicy",
        env=env,
        verbose=1,
    )

In [2]:
file = qb.Download("https://www.dropbox.com/s/dilyxfjvhuf8hff/TSLA_A2C_641k_641150_iter3.zip?dl=1")

In [46]:
files = {'data': qb.Download("https://www.dropbox.com/s/7kqsi9v6c8oj39z/data?dl=1"),
        'parameter_list': qb.Download("https://www.dropbox.com/s/32h4ogfbtsojlmn/parameter_list?dl=1"),
        'parameters': qb.Download("https://www.dropbox.com/s/vrdcvrm7sereml2/parameters?dl=1")}

In [1]:
import json
qb = QuantBook()
file = json.loads(qb.Download('https://raw.githubusercontent.com/iamianM/model/main/parameter_arrays/filenames.json'))

In [14]:
parameters = {}
for f, shape in file.items():
    d = qb.Download(f'https://raw.githubusercontent.com/iamianM/model/main/parameter_arrays/{f}')
    parameters[f.replace('_', '/').replace('-', ':')[:-4]] = np.fromstring(d, sep=' ').reshape(shape)

In [15]:
model.load_parameters(parameters)

In [20]:
val_env_a2c = StockTradingEnv(
    df,
    model_name='TSLA',
    mode='validation',
    terminal=True,
    print_verbosity=100,
    hmax= 1.0, 
    initial_amount= 50000, 
    buy_cost_pct= lambda x: 0.01, 
    sell_cost_pct= lambda x: 0.01, 
    tech_indicator_list= [],  
    get_gif=False,
)

In [21]:
val_obs_a2c = val_env_a2c.reset()
for i in range(len(val_env_a2c.df.index.unique())):
    action, _states = model.predict(val_obs_a2c)
    test_obs, rewards, dones, info = val_env_a2c.step(action)
    if dones:
        print("hit end!")
        break